In [1]:
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt
import torchvision
import torch
import os
import random
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, roc_curve, plot_roc_curve, auc
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
from torch.utils.data import DataLoader
import pickle
from sklearn.metrics import accuracy_score, f1_score
import torch.utils.data as data_utils
from collections import defaultdict
import gc

In [2]:
def load_and_transform_cifar100(label_noise=0.0, seed=1234):
    training_data = datasets.CIFAR100(
        root="data",
        train=True,
        download=True,
    )

    test_data = datasets.CIFAR100(
        root="data",
        train=False,
        download=True,
    )    
        
    
    if label_noise > 0:
        np.random.seed(seed)
        size = int(label_noise * len(training_data))
        noise_idx = set(np.random.choice(len(training_data), size, replace=False))
        print('injecting noise ...%s' %len(noise_idx))
        targets = []
        for idx, t in enumerate(training_data.targets):
            if idx in noise_idx:
                targets.append(np.random.randint(100))
            else:
                targets.append(t)
                
        training_data.targets = targets
    
    
        
    image_size = (128, 128)
    resnet_mean = [0.485, 0.456, 0.406]
    resnet_std = [0.229, 0.224, 0.225]

    #Creating a Transformation Object
    data_transform = torchvision.transforms.Compose([
        torchvision.transforms.Resize(size=image_size),
        torchvision.transforms.ToTensor(), #Converting to tensor
        torchvision.transforms.Normalize(mean = resnet_mean,
                                        std = resnet_std) 
    ])
    training_data.transform = data_transform
    test_data.transform = data_transform
    return training_data, test_data

In [3]:
#creates original cifar coarse to fine and back dictionaries
 
class Dictlist(dict):
    def __setitem__(self, key, value):
        try:
            self[key]
        except KeyError:
            super(Dictlist, self).__setitem__(key, [])
        self[key].append(value)
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dic = pickle.load(fo, encoding='bytes')
    return dic
 
x=unpickle('data/cifar-100-python/test')
d=Dictlist()
 
for i in range(0,len(x[b'coarse_labels'])):
    d[x[b'coarse_labels'][i]]=x[ b'fine_labels'][i]
coarse_to_fine=dict(d)
for i in coarse_to_fine.keys():
    coarse_to_fine[i]=list(dict.fromkeys(coarse_to_fine[i]))

fine_to_coarse = dict()
for k,v in coarse_to_fine.items():
    for each in v:
        fine_to_coarse[each] = k

In [4]:
def filter_data(training_data, test_data, filter_by_coarse=False, 
                coarse_label=None, coarse_to_fine=None, valid_size=0.1, batch_size=64, data_fraction=1.0, seed=1234):
    """Filters datasets by coarse label if needed and split to train and validation subsets"""

    np.random.seed(seed)
    size = int(data_fraction * len(training_data))
    sample_idx = np.random.choice(len(training_data), size, replace=False)
    print('sampling data ...%s' %len(sample_idx))
    
    if filter_by_coarse:
        labels = coarse_to_fine[coarse_label]
        indices = [idx for idx,v in enumerate(training_data.targets) if v in labels and idx in sample_idx]
        indices_test = [idx for idx,v in enumerate(test_data.targets) if v in labels]     
    else:
        indices = sample_idx
        indices_test = [i for i in range(len(test_data.targets))]

        
    
    validate_len = int(len(indices)*valid_size)
    validate_filter_indices = random.sample(range(0,len(indices)), validate_len)
    indices_val = [indices[i] for i in validate_filter_indices]  
    indices_train = [v for i,v in enumerate(indices) if i not in validate_filter_indices]    
        
    train = data_utils.Subset(training_data, indices_train)
    validate = data_utils.Subset(training_data, indices_val)
    test = data_utils.Subset(test_data, indices_test)
    
    train_dl = DataLoader(train,  batch_size=batch_size, shuffle=True)
    validate_dl = DataLoader(validate, batch_size=batch_size, shuffle=True)
    test_dl = DataLoader(test,  batch_size=batch_size, shuffle=True)
    return train_dl, validate_dl, test_dl

In [5]:
def create_label_dictionaries(cluster_list):
    """Creates dictionaries from coarse to fine, from fine to coarse and concatenated list 
    of fine clusters
    
    Parameters:
    cluster_list: list
        A list where the index of the element represents the fine label and the value represents
        the coarse label(cluster)    
    """
    fine_to_coarse = {i:v for i,v in enumerate(cluster_list)}
    coarse_to_fine = defaultdict(list)
    for k, v in fine_to_coarse.items():
        coarse_to_fine[v].append(k)
    return fine_to_coarse, coarse_to_fine




# #creates original cifar coarse to fine and back dictionaries

# class Dictlist(dict):
#     def __setitem__(self, key, value):
#         try:
#             self[key]
#         except KeyError:
#             super(Dictlist, self).__setitem__(key, [])
#         self[key].append(value)

# def unpickle(file):
#     import pickle
#     with open(file, 'rb') as fo:
#         dic = pickle.load(fo, encoding='bytes')
#     return dic


# x=unpickle('data/cifar-100-python/test')
# d=Dictlist()

# for i in range(0,len(x[b'coarse_labels'])):
#     d[x[b'coarse_labels'][i]]=x[ b'fine_labels'][i]
# coarse_to_fine=dict(d)
# for i in coarse_to_fine.keys():
#     coarse_to_fine[i]=list(dict.fromkeys(coarse_to_fine[i]))
    
# fine_to_coarse = dict()
# for k,v in coarse_to_fine.items():
#     for each in v:
#         fine_to_coarse[each] = k
    

In [6]:
class Resnext50(torch.nn.Module):
    def __init__(self, n_classes, name):
        super().__init__()
        self.name = name
        resnet = torchvision.models.resnext50_32x4d(pretrained=True, progress=True)
        resnet.fc = torch.nn.Sequential(
            torch.nn.Dropout(p=0.2),
            torch.nn.Linear(in_features=resnet.fc.in_features, out_features=n_classes)
        )
        self.base_model = resnet
        self.soft = torch.nn.Softmax()

    def forward(self, x):
        return self.soft(self.base_model(x))   
    

In [7]:
def train_loop(dataloader, model, loss_fn, optimizer, mode="flat", lookup=[], fine_to_coarse={}, batch_size=64):
    size = len(dataloader.dataset)
    model.train()
    train_loss = 0
    counter = 0
    with tqdm(dataloader, unit="batch") as tepoch:
        counter += 1
        for X_cpu, y_cpu in tepoch:
            if mode == "fine":
                y_cpu = torch.tensor([lookup.index(i) for i in y_cpu])
            elif mode == "coarse":
                y_cpu = torch.tensor([fine_to_coarse[int(target)] for target in y_cpu])
            X, y = X_cpu.to(device), y_cpu.to(device)
            optimizer.zero_grad()
            outputs = model(X)
            loss = loss_fn(outputs, y.type(torch.long))
            loss.backward() 
            optimizer.step() 
            train_loss += loss.item() 
            avg_loss = train_loss / (counter * batch_size)
            tepoch.set_postfix(loss=avg_loss)
    return avg_loss
    
    
def eval_model(dataloader, model, mode="flat", lookup=[], fine_to_coarse={}, return_probab=False):
    model.eval()
    predicts = []
    targets = []
    for counter, (images, labels) in enumerate(dataloader):
        if mode == "fine":
            labels = [lookup.index(i) for i in labels]
        elif mode == "coarse":
            labels = [fine_to_coarse[int(i)] for i in labels]
        imageGPU = images.to(device)
        
        outputs = torch.Tensor.cpu(model(imageGPU))
        predicts.append(outputs.detach().numpy())
        targets.append(labels)
        
    predicts = np.vstack(predicts)
    targets = np.hstack(targets)
    if return_probab:
        return predicts, targets
    else:
        predicts = np.argmax(predicts, axis=1)
        return accuracy_score(targets, predicts)
    


In [8]:
def train_and_validate(epochs, model, loss_fn, optimizer, train_dl, val_dl, test_dl, 
                       mode="flat", fine_to_coarse={}, coarse_label=None, lookup=[], model_path = None):
    if model_path is None:
        model_path = '/models/%s.pth' %model.name
    min_loss= 1000
    max_acc = 0
    for t in range(epochs):
        if mode == "fine":
            print(f"Coarse label {coarse_label} Epoch {t+1}\n-------------------------------")
        else:
            print(f"Epoch {t+1}\n-------------------------------")
        train_loss = train_loop(train_dl, model, loss_fn, optimizer, mode=mode, fine_to_coarse=fine_to_coarse, lookup=lookup)
        val_acc = eval_model(val_dl, model, mode=mode, fine_to_coarse=fine_to_coarse, lookup=lookup)
        if train_loss < min_loss:
            max_acc = val_acc
            min_loss = train_loss
            torch.save(model, model_path)
        elif train_loss == min_loss and val_acc>max_acc:
            max_acc = val_acc
            min_loss = train_loss
            torch.save(model, model_path)
        print("validation acc %s and loss %s" %(val_acc, train_loss))
        test_acc = eval_model(test_dl, model, mode=mode, fine_to_coarse=fine_to_coarse, lookup=lookup)
        print("done epoch %s : test_acc %s" %(t, test_acc))
    print("Done! Saved model with validation accuracy %s and loss %s" %(max_acc, min_loss))
    return (max_acc, test_acc)

In [34]:
#load clusters data
tasksim_clusters = pickle.load(open('/home/kalytv/teamdrive/transmediasp/kate/cifar_tasksim_clusters_1.pkl', 'rb'))
tasksim_clusters

[array([11,  6,  7, 15,  3,  2,  5, 17,  2, 16, 23,  7, 12, 22,  6,  4, 18,
        12,  6, 13,  0,  3, 18,  8,  6,  0,  6,  3, 18,  4, 20,  4, 21,  1,
        14,  7,  3, 12, 15, 18, 23,  2,  3, 14,  6,  6,  7,  1, 22,  8, 15,
         4, 19, 11,  5,  3, 13, 23,  9,  1,  1, 23,  5, 15, 15, 15, 15, 21,
        12, 12,  5,  8, 20, 10, 15, 15, 12,  6,  6,  6, 15,  9,  5, 23,  0,
        12, 18,  0, 15,  2, 12, 21, 24, 21,  0, 20,  1, 14,  7,  6])]

In [36]:
acorn = 1234
torch.manual_seed(acorn)
np.random.seed(acorn)

torch.cuda.is_available()

if torch.cuda.is_available():  
    dev = "cuda" 
else:  
    dev = "cpu"  
    
device = torch.device(dev)  

batch_size = 64
valid_size = 0.1

In [37]:
#Prepare data for a coarse classifier training

fine_to_coarse, coarse_to_fine = create_label_dictionaries(tasksim_clusters[0])
print(coarse_to_fine)
number_of_clusters = len(coarse_to_fine.keys())
overall_accuracy={}

stacked_fine_labels = []
for i in range(0, number_of_clusters):
    stacked_fine_labels +=coarse_to_fine[i]

training_data, test_data=load_and_transform_cifar100(seed=acorn)
train_dl, validate_dl, test_dl = filter_data(training_data, test_data, valid_size=valid_size, 
                                             batch_size=batch_size,  data_fraction=1, seed=acorn)

#coarse classifier training

learning_rate = 1e-4
epochs = 15
model = Resnext50(number_of_clusters, name="hierarchical_coarse_model_Resnext50_25comp")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

model.to(device)
loss_fn = torch.nn.CrossEntropyLoss().to(device)

coarse_max_acc, coarse_test_acc =train_and_validate(epochs, model, loss_fn, optimizer, train_dl, validate_dl, test_dl, 
                                                    mode="coarse", fine_to_coarse=fine_to_coarse,
                                                    model_path = 'models/hierarchical/%s.pth' %model.name )
print("coarse accuracy %s" %coarse_test_acc)

defaultdict(<class 'list'>, {11: [0, 53], 6: [1, 14, 18, 24, 26, 44, 45, 77, 78, 79, 99], 7: [2, 11, 35, 46, 98], 15: [3, 38, 50, 63, 64, 65, 66, 74, 75, 80, 88], 3: [4, 21, 27, 36, 42, 55], 2: [5, 8, 41, 89], 5: [6, 54, 62, 70, 82], 17: [7], 16: [9], 23: [10, 40, 57, 61, 83], 12: [12, 17, 37, 68, 69, 76, 85, 90], 22: [13, 48], 4: [15, 29, 31, 51], 18: [16, 22, 28, 39, 86], 13: [19, 56], 0: [20, 25, 84, 87, 94], 8: [23, 49, 71], 20: [30, 72, 95], 21: [32, 67, 91, 93], 1: [33, 47, 59, 60, 96], 14: [34, 43, 97], 19: [52], 9: [58, 81], 10: [73], 24: [92]})
Files already downloaded and verified
Files already downloaded and verified
sampling data ...50000


  0%|          | 0/704 [00:00<?, ?batch/s]

Epoch 1
-------------------------------


/anaconda/envs/py37_default/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  
100%|██████████| 704/704 [04:14<00:00,  2.77batch/s, loss=29.5]


validation acc 0.6976 and loss 29.475340638309717


...

validation acc 0.7336 and loss 27.26087200269103


  0%|          | 0/704 [00:00<?, ?batch/s]

done epoch 13 : test_acc 0.7296
Epoch 15
-------------------------------


100%|██████████| 704/704 [04:01<00:00,  2.91batch/s, loss=27.3]


validation acc 0.7418 and loss 27.261237166821957
done epoch 14 : test_acc 0.7426
Done! Saved model with validation accuracy 0.7336 and loss 27.26087200269103
coarse accuracy 0.7426


In [38]:
#fine clusters training

learning_rate = 1e-4
epochs = 15
accuracies = {}


for coarse_label in range(0, number_of_clusters):
        if len(coarse_to_fine[coarse_label])>1:
            train_dl, val_dl, test_dl =  filter_data(training_data, test_data, filter_by_coarse=True, 
                    coarse_label=coarse_label, coarse_to_fine=coarse_to_fine, valid_size=0.1, 
                    batch_size=batch_size, data_fraction=1, seed=acorn)
            # Initialize the model
            model = Resnext50(len(coarse_to_fine[coarse_label]), name="hierarchical_fine_model_coarse_label_%s_25comp" %coarse_label)
            optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
            model.to(device)
            loss_fn = torch.nn.CrossEntropyLoss().to(device)
            max_acc, test_acc = train_and_validate(epochs, model, loss_fn, optimizer, train_dl, val_dl, test_dl, mode="fine", 
                                                   coarse_label=coarse_label, lookup=coarse_to_fine[coarse_label], 
                                                   model_path = 'models/hierarchical/%s.pth' %model.name )
            accuracies[coarse_label] = (max_acc, test_acc)
print(accuracies)

sampling data ...50000


  0%|          | 0/36 [00:00<?, ?batch/s]

Coarse label 0 Epoch 1
-------------------------------


/anaconda/envs/py37_default/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  
100%|██████████| 36/36 [00:13<00:00,  2.70batch/s, loss=0.661]


validation acc 0.904 and loss 0.6612682621926069


...

validation acc 0.9 and loss 0.5169124472886324


  0%|          | 0/36 [00:00<?, ?batch/s]

done epoch 13 : test_acc 0.908
Coarse label 23 Epoch 15
-------------------------------


100%|██████████| 36/36 [00:11<00:00,  3.00batch/s, loss=0.521]


validation acc 0.896 and loss 0.5211138455197215
done epoch 14 : test_acc 0.89
Done! Saved model with validation accuracy 0.9 and loss 0.5169124472886324
{0: (0.9, 0.93), 1: (0.824, 0.828), 2: (0.955, 0.9525), 3: (0.87, 0.905), 4: (0.95, 0.935), 5: (0.932, 0.926), 6: (0.8927272727272727, 0.8681818181818182), 7: (0.616, 0.642), 8: (0.8866666666666667, 0.9166666666666666), 9: (0.99, 0.95), 11: (0.98, 0.95), 12: (0.8675, 0.91875), 13: (1.0, 1.0), 14: (0.9533333333333334, 0.9533333333333334), 15: (0.8654545454545455, 0.8209090909090909), 18: (0.932, 0.92), 20: (0.8533333333333334, 0.8533333333333334), 21: (0.865, 0.8875), 22: (0.98, 0.98), 23: (0.9, 0.89)}


In [39]:
accuracies

{0: (0.9, 0.93),
 1: (0.824, 0.828),
 2: (0.955, 0.9525),
 3: (0.87, 0.905),
 4: (0.95, 0.935),
 5: (0.932, 0.926),
 6: (0.8927272727272727, 0.8681818181818182),
 7: (0.616, 0.642),
 8: (0.8866666666666667, 0.9166666666666666),
 9: (0.99, 0.95),
 11: (0.98, 0.95),
 12: (0.8675, 0.91875),
 13: (1.0, 1.0),
 14: (0.9533333333333334, 0.9533333333333334),
 15: (0.8654545454545455, 0.8209090909090909),
 18: (0.932, 0.92),
 20: (0.8533333333333334, 0.8533333333333334),
 21: (0.865, 0.8875),
 22: (0.98, 0.98),
 23: (0.9, 0.89)}

In [40]:
#Evaluation of the whole model
    
#loading models
gc.collect()
torch.cuda.empty_cache()
model_coarse = torch.load('models/hierarchical/hierarchical_coarse_model_Resnext50_25comp.pth')
fine_models = {}
for i in range(0,number_of_clusters):
    if len(coarse_to_fine[i])>1:
        fine_models['fine_model_' + str(i)] = torch.load('models/hierarchical/hierarchical_fine_model_coarse_label_%s_25comp.pth' %i)
test_batch_dl = DataLoader(test_data,  batch_size=batch_size, shuffle=False)


In [41]:
coarse_predicts = []
targets = []
model_coarse.to(device)
model_coarse.eval()
for counter, (images, labels) in enumerate(test_batch_dl):
    imageGPU = images.to(device)
    outputs = torch.Tensor.cpu(model_coarse(imageGPU))
    coarse_predicts.append(outputs.detach().numpy())
    targets.append(labels)
targets = np.hstack(targets)


/anaconda/envs/py37_default/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


In [42]:
#reformat coarse prediction
coarse_prediction = np.vstack(coarse_predicts)

column_num = 0
for i in range(0, number_of_clusters):
    cluster_len = len(coarse_to_fine[i])
    for rep in range(cluster_len-1):
        coarse_prediction = np.insert(coarse_prediction, column_num, coarse_prediction[:,column_num], axis=1)
    column_num +=cluster_len

In [43]:
fine_prediction= []

for i in range(0,number_of_clusters):
    if len(coarse_to_fine[i]) == 1:
        fine_prediction.append(np.ones((len(coarse_prediction),1)))
    else:
        predicts = []
        print("Evaluating fine model %s" %i)
        fine_model = fine_models['fine_model_%s' %i]
        fine_model.to(device)
        fine_model.eval()
        for counter, (images, labels) in enumerate(test_batch_dl):
            imageGPU = images.to(device)
            outputs = torch.Tensor.cpu(fine_model(imageGPU))
            predicts.append(outputs.detach().numpy())   
        predicts = np.vstack(predicts)
        fine_prediction.append(predicts)

Evaluating fine model 0


/anaconda/envs/py37_default/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


Evaluating fine model 1
Evaluating fine model 2
Evaluating fine model 3
Evaluating fine model 4
Evaluating fine model 5
Evaluating fine model 6
Evaluating fine model 7
Evaluating fine model 8
Evaluating fine model 9
Evaluating fine model 11
Evaluating fine model 12
Evaluating fine model 13
Evaluating fine model 14
Evaluating fine model 15
Evaluating fine model 18
Evaluating fine model 20
Evaluating fine model 21
Evaluating fine model 22
Evaluating fine model 23


In [44]:
#reformat fine_prediction
fine_prediction = np.hstack(fine_prediction)
result = np.multiply(fine_prediction, coarse_prediction)
result = np.argmax(result, axis=1)
result_decoded = [stacked_fine_labels[i] for i in result]
final_acc = accuracy_score(targets, result_decoded) 
print("overall accuracy is %s" %final_acc)

overall accuracy is 0.6555


In [49]:
accuracy_score(targets, result_decoded)

0.6555